Q)Find the frequent items from the given dataset.
Tid:-1,2,3,4,5

Items:{milk,butter,bread},{butter,bread},{milk,bread},{milk,butter},{milk,coke}

Apply FP-growth algorithm.Find association rules with minimu support=3 and confidence=68%

In [ ]:
#first install 
#pip install mlxtend
from collections import defaultdict, deque
import itertools

class FPNode:
    def __init__(self, item, count, parent=None):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.next = None

class FPTree:
    def __init__(self):
        self.root = FPNode(item=None, count=0)
        self.header_table = defaultdict(list)

    def add_transaction(self, transaction):
        current_node = self.root
        for item in transaction:
            if item not in current_node.children:
                new_node = FPNode(item=item, count=1, parent=current_node)
                current_node.children[item] = new_node
                self.header_table[item].append(new_node)
            else:
                current_node.children[item].count += 1
            current_node = current_node.children[item]

    def mine_patterns(self, min_support):
        frequent_itemsets = []
        self._mine_patterns(self.root, min_support, [], frequent_itemsets)
        return frequent_itemsets

    def _mine_patterns(self, node, min_support, current_pattern, frequent_itemsets):
        if node.item is not None and node.count >= min_support:
            new_pattern = current_pattern + [node.item]
            frequent_itemsets.append((tuple(new_pattern), node.count))
            for child in node.children.values():
                self._mine_patterns(child, min_support, new_pattern, frequent_itemsets)
    
def create_fp_tree(transactions, min_support):
    tree = FPTree()
    item_counts = defaultdict(int)
    
    for transaction in transactions:
        for item in transaction:
            item_counts[item] += 1
    
    # Filter out items that do not meet the minimum support
    item_counts = {item: count for item, count in item_counts.items() if count >= min_support}
    
    # Sort items by frequency and alphabetically
    sorted_items = sorted(item_counts.keys(), key=lambda x: (-item_counts[x], x))
    
    # Create FP-Tree
    for transaction in transactions:
        filtered_transaction = [item for item in transaction if item in item_counts]
        filtered_transaction.sort(key=lambda x: sorted_items.index(x))
        tree.add_transaction(filtered_transaction)
    
    return tree

def main():
    transactions = [
        {'milk', 'butter', 'bread'},
        {'butter', 'bread'},
        {'milk', 'bread'},
        {'milk', 'butter'},
        {'milk', 'coke'}
    ]
    
    min_support = 3
    tree = create_fp_tree(transactions, min_support)
    
    # Mine frequent itemsets
    frequent_itemsets = tree.mine_patterns(min_support)
    
    print("Frequent Itemsets:")
    for itemset, support in frequent_itemsets:
        print(f"{itemset}: {support}")


main()


In [ ]:
from mlxtend.frequent_patterns import fpgrowth
import pandas as pd

# Define the dataset
transactions = [
    {'milk', 'butter', 'bread'},
    {'butter', 'bread'},
    {'milk', 'bread'},
    {'milk', 'butter'},
    {'milk', 'coke'}
]

# Convert dataset to one-hot encoded DataFrame
all_items = sorted(set(item for sublist in transactions for item in sublist))
df = pd.DataFrame([{item: (item in transaction) for item in all_items} for transaction in transactions])

# Apply FP-Growth algorithm
frequent_itemsets = fpgrowth(df, min_support=3/5, use_colnames=True)
print(frequent_itemsets)
